In [1]:
import pandas as pd
import seaborn as sns
import pylab as plt
import pandas as pd
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC  
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By 
import time
import requests
import warnings
from bs4 import BeautifulSoup
from tqdm import tqdm
warnings.filterwarnings('ignore')
from multiprocessing import Pool
from fake_useragent import UserAgent
import requests
from multiprocessing import Pool, cpu_count
from joblib import Parallel, delayed
from tqdm_joblib import tqdm_joblib


In [2]:
film = pd.read_csv('../data/Net_titles.csv', encoding='utf-8', encoding_errors='ignore')
actor = pd.read_csv('../data/Net_credits.csv', encoding='utf-8', encoding_errors='ignore') 

In [3]:
# Para rellenar los valores nulos de la columna Age_certification. Si pertenece a los siguientes géneros se asumirá que la certificación de edad es R (+17)
def fill_age_null(row):
    if pd.isna(row['age_certification']):
        if 'thriller' in row['genres'] or 'horror' in row['genres'] or 'black comedy' in row['genres']:
            return 'R'
    return row['age_certification']

film['age_certification'] = film.apply(lambda row: fill_age_null(row), axis=1)

In [4]:
# Eliminamos los valores nulos de las columnas relacionados con IMDB y TMDB, ya que sin el imdb_id es imposible analizarlos
# Rellenamos los valores nulos de la columna age_certification con 'unknown', y los de la columna seasons con '-1'
# Rellenamos los valores nulos de la columna character del df 'actor' con la palabra 'unknown'
# Rellenamos los valores nulos de la columna description con 'unknown'

film.dropna(subset=['imdb_id', 'imdb_score', 'imdb_votes', 'tmdb_popularity', 'tmdb_score'], inplace=True)
film['age_certification'] = film['age_certification'].fillna('unknown')
film['seasons'] = film['seasons'].fillna('-1')
film['description'] = film['description'].fillna('unknown')
actor['character'] = actor['character'].fillna('unknown')



In [5]:
film = film.drop(film.index[200:])

In [37]:
reviews = []

for id in tqdm(film['imdb_id']):
    url = f'https://www.imdb.com/title/{id}/reviews/?ref_=tt_ov_rt'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Encontrar todos los títulos de las reseñas en la página actual y agregarlos a la lista de reseñas
    a = soup.find_all('a', {'class': 'title'})
    for e in a:
        if e.text.strip() == '':
            rating_title = None  # Si no hay valoración, agregar un valor nulo
        else:
            reviews_title = e.text.strip()
            reviews.append((id, reviews_title))
com_rev = pd.DataFrame(reviews, columns=['id', 'review_title'])




IndentationError: expected an indented block (1165594241.py, line 6)

In [6]:
rating = []

for id in tqdm(film['imdb_id']):
    url = f'https://www.imdb.com/title/{id}/reviews/?ref_=tt_ov_rt'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Lo mismo con las valoraciones
    a = soup.find_all('span', {'class': 'rating-other-user-rating'})
    for e in a:
        if e.text.strip() == '':
            rating_title = None  # Si no hay valoración, agregar un valor nulo
        else:
            rating_title = e.text.strip()
            rating.append((id, rating_title))

com_rat = pd.DataFrame(rating, columns=['id', 'rating_title'])

  0%|          | 12/5131 [00:17<2:02:22,  1.43s/it]


KeyboardInterrupt: 

In [ ]:
#proceso sobre todas las películas

comentarios = {'imdb_id': [], 'comentarios': []}        #diccionario vacío donde almacenaremos los comentarios de las películas

for i in tqdm(range(len(film))):
    imdb_id = film.loc[i, 'imdb_id']        # Obtenemos el ID de la película
    if pd.isna(imdb_id):                    # Si el ID es nulo, saltamos a la siguiente fila
        print(f'IMDB ID nulo en fila {i}, saltando a la siguiente película')
        continue
    url = f'https://www.imdb.com/title/{imdb_id}/reviews?ref_=tt_ov_rt'     # Creamos la URL de los comentarios de la película usando su ID
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        reviews = soup.find_all('div', {'class': 'text show-more__control'})        #buscamos los elementos sobre los que queremos iterar
        ratings = soup.find_all('span', {'class': 'rating-other-user-rating'})
        comentarios_pelicula = {'imdb_id': imdb_id, 'comentarios': []}
        for i in range(len(reviews)):
            review_div = reviews[i].find_parent('div', {'class': 'lister-item-content'})        # Obtenemos comentarios y calificación
            review = review_div.find('a', {'class': 'title'}).text.strip() if review_div.find('a', {'class': 'title'}) else 'No title'
            rating = ratings[i].text.strip() if len(ratings) > i else 'Sin calificación'
            comentario = {'review': review, 'rating': rating}           #creamos un diccionario para almacenar comentario y calificación
            comentarios_pelicula['comentarios'].append(comentario)      #Añadimos el comentario al diccionario de comentarios
            comentarios['imdb_id'].append(imdb_id)
            comentarios['comentarios'].append(comentarios_pelicula)
            #print(f'Comentarios de película con IMDB ID {imdb_id} obtenidos')
    else:
        print(f'Error al obtener comentarios de la película con IMDB ID {imdb_id}')

df_comentarios = pd.DataFrame(comentarios)
df_comentarios.to_csv('comentarios_peliculas.csv', index=False)

In [9]:
from joblib import Parallel, delayed
from tqdm import tqdm

def extract_reviews_and_ratings(imdb_id):
    url = f'https://www.imdb.com/title/{imdb_id}/reviews?ref_=tt_ov_rt' 
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        reviews = soup.find_all('div', {'class': 'text show-more__control'})  
        ratings = soup.find_all('span', {'class': 'rating-other-user-rating'})
        results = []
        for rev, rat in zip(reviews, ratings):
            rev_div = rev.find_parent('div', {'class': 'lister-item-content'})  
            rev_title = rev_div.find('a', {'class': 'title'}).text.strip() if rev_div.find('a', {'class': 'title'}) else 'No title'
            rat_title = rat.text.strip() if rat else 'Sin calificación'
            results.append((imdb_id, rev_title, rat_title))
        return results
    else:
        print(f'Error al obtener comentarios de la película con IMDB ID {imdb_id}')
        return []

if __name__ == '__main__':
    imdb_ids = film['imdb_id'].tolist()
    reviews_and_ratings = Parallel(n_jobs=-1)(delayed(extract_reviews_and_ratings)(imdb_id) for imdb_id in tqdm(imdb_ids))
    reviews_and_ratings = [review for sublist in reviews_and_ratings for review in sublist]
    com_rev = pd.DataFrame(reviews_and_ratings, columns=['id', 'review_title', 'rating_title'])


100%|██████████| 200/200 [00:34<00:00,  5.83it/s]


In [10]:
com_rev

,id,review_title,rating_title
0,tt0075314,A classy character study of a disturbed indivi...,7/10
1,tt0075314,A wonderfully engaging and convincing slide in...,10/10
2,tt0075314,Ladies and gentlemen: Mr. Robert De Niro!,10/10
3,tt0075314,Still has the power to shock...,10/10
4,tt0075314,"Disturbing, powerful, relevant, important",9/10
...,...,...,...
2745,tt0112916,Mismash,3/10
2746,tt0112916,Sunny Deol wasted by terrible direction!,5/10
2747,tt0112916,Usual Hindi mas ala...,5/10
2748,tt0112916,Good movie,7/10
